# Introduction
**Objective**:<br>
This notebook is meant to do data validation using `Great Expectation` to ensure the clean data follow these criteria: 
- To be unique
- To be between a range of value 
- To be in set 
- To be in type list 
- To be in specific format
- To be existed
- Columns count to be equal to



# Load Library 

In [1]:
from great_expectations.data_context import FileDataContext

# Connect to Datasourse

In [2]:
# Initialize the Great Expectations context
context = FileDataContext.create(project_root_dir='./')

In [3]:
# unique name for the datasource
datasource_name = 'data'
datasource = context.sources.add_pandas(name=datasource_name)

In [4]:
# Define a name and path for the data asset
asset_name = 'fraud_data_clean'
path_to_data = '/Users/nathanaelh/Desktop/untitled folder/dags/fraud_data_clean.csv'

# Add the CSV asset to the datasource
asset = datasource.add_csv_asset(name=asset_name, filepath_or_buffer=path_to_data)

In [5]:
# Build a batch request for the asset
batch_request = asset.build_batch_request()

# Create Expectation Suite 

In [6]:
# name for the expectation suite
expectation_suite_name = 'fraud_clean'

# Add or update the expectation suite in the context
context.add_or_update_expectation_suite(expectation_suite_name)

{
  "expectation_suite_name": "fraud_clean",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.16"
  }
}

In [7]:
# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name
)

# Display the first few rows of the data to check the validator
print(validator.head())

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

  transaction_id        date day_of_week  time type_of_card entry_mode  \
0      #3577 209  2020-10-14   Wednesday    19         Visa        Tap   
1      #3039 221  2020-10-14   Wednesday    17   MasterCard        PIN   
2      #2694 780  2020-10-14   Wednesday    14         Visa        Tap   
3      #2640 960  2020-10-13     Tuesday    14         Visa        Tap   
4      #2771 031  2020-10-13     Tuesday    23         Visa        CVC   

   amount type_of_transaction merchant_group country_of_transaction  \
0     5.0                 POS  Entertainment         United Kingdom   
1   288.0                 POS       Services                    USA   
2     5.0                 POS     Restaurant                  India   
3    28.0                 POS  Entertainment         United Kingdom   
4    91.0              Online    Electronics                    USA   

  shipping_address country_of_residence  gender  age      bank       fraud  
0   United Kingdom       United Kingdom    Male   2

In [8]:
# Creat an expectation suite
expectation_suite_name = 'P2M3_devi_nirfana_clean'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,transaction_id,date,day_of_week,time,type_of_card,entry_mode,amount,type_of_transaction,merchant_group,country_of_transaction,shipping_address,country_of_residence,gender,age,bank,fraud
0,#3577 209,2020-10-14,Wednesday,19,Visa,Tap,5.0,POS,Entertainment,United Kingdom,United Kingdom,United Kingdom,Male,25,RBS,Legitimate
1,#3039 221,2020-10-14,Wednesday,17,MasterCard,PIN,288.0,POS,Services,USA,USA,USA,Female,49,Lloyds,Legitimate
2,#2694 780,2020-10-14,Wednesday,14,Visa,Tap,5.0,POS,Restaurant,India,India,India,Female,42,Barclays,Legitimate
3,#2640 960,2020-10-13,Tuesday,14,Visa,Tap,28.0,POS,Entertainment,United Kingdom,India,United Kingdom,Female,51,Barclays,Legitimate
4,#2771 031,2020-10-13,Tuesday,23,Visa,CVC,91.0,Online,Electronics,USA,USA,United Kingdom,Male,38,Halifax,Fraud


## 1. Expectation to be unique 

In [9]:
validator.expect_compound_columns_to_be_unique(['transaction_id', 'date', 'day_of_week', 'time', 'type_of_card', 'entry_mode', 'amount'])

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 99977,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The output show `"success" : true` which mean the compound columns of `transaction_id`, `date`, `day_of_week`, `time`, `type_of_card`, `entry_mode` and `amount` are unique.

There is a reason why I do not use `transaction_id` alone to ensure column uniqueness. It is because there is a possibility of a transaction being split into multiple parts, resulting in the same `transaction_id` appearing more than once (e.g. for split payments transactions).

## 2. Expectation to be between a range of value 

In [10]:
validator.expect_column_values_to_be_between(column="age", min_value=15, max_value=90)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 99977,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The validation is done to ensure the integrity and accuracy of the dataset by verifying that all age values fall within the expected range of 15 to 90 years.

- Many credit card issuers allow individuals as young as **15 years old to be added as authorized users** on their parents' or guardians' credit card accounts. This validation helps ensure that the dataset includes only those who are legally eligible to have a credit card or be an authorized user.
- **The upper limit of 90 years** is set to cover the typical lifespan during which individuals actively hold and use credit cards. While it is less common for individuals above 90 years old to hold credit cards, setting this upper limit helps account for older adults who may still be active credit card users.

## 3. Expectation to be in set

In [11]:
validator.expect_column_values_to_be_in_set(column="bank", value_set=['RBS', 'Lloyds', 'Barclays', 'Halifax', 'Monzo', 'HSBC', 'Metro'])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 99977,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The validation was successful, indicating that all values in the `bank` column of the dataset are correctly limited to the predefined set of valid bank names: `RBS`, `Lloyds`, `Barclays`, `Halifax`, `Monzo`, `HSBC`, and `Metro`.

## 4. Expectation to be in type list

In [12]:
validator.expect_column_values_to_be_in_type_list("amount", ['int64', 'float64'])


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The validation test for the `amount` column's data type was successful. The observed data type is `float64,` which aligns with the expected types of either 'int64' or 'float64'. No exceptions were raised during the validation process. This indicates that the column's values conform to the specified data type requirements, ensuring consistency and accuracy in the dataset.

## 5. Date to be in specific format

In [13]:
validator.expect_column_values_to_match_strftime_format("date", strftime_format="%Y-%m-%d")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 99977,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The validation test for the `date` column's format was successful. Among 99,977 elements, none were found to be unexpected or not conforming to the specified `strftime` format `%Y-%m-%d`. This indicates that all values in the `date` column adhere to the expected format, ensuring consistency and reliability in the dataset. No exceptions were raised during the validation process, affirming the integrity of the data.

## 6. Columns amount to exist

In [14]:
validator.expect_column_to_exist('amount')

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "result": {},
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The validation test for the existence of the `amount` column was successful, indicating that the column does indeed exist within the dataset. No exceptions were raised during the validation process. 

The presence of the `amount` column is crucial in transactional data as it typically represents the monetary value associated with each transaction. This column holds essential information regarding the financial aspect of transactions, such as purchase amounts, fees, or currency exchange rates. Without the `amount` column, analyzing and understanding the financial aspects of the transactions would be significantly hindered, making it a fundamental component in transactional datasets.

## 7. Columns count to be equal to 16

In [15]:
validator.expect_table_column_count_to_equal(16)

Calculating Metrics:   0%|          | 0/3 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 16
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The validation test for the column count in the dataset was successful, with the observed value matching the expected count of 16 columns. This indicates that there has been no change in the number of columns after both the data cleaning and pre-cleaning processes, remaining consistent at 16 columns. 

Maintaining the same number of columns underscores the effectiveness of the data cleaning process in preserving the structural integrity of the dataset. It suggests that the cleaning procedures likely focused on refining data quality within the existing framework rather than altering the dataset's structure.

# Checkpoint 

In [16]:
# Save into Expectation Suite
validator.save_expectation_suite(discard_failed_expectations=False)

In [17]:
# Create a checkpoint
checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_1',
    validator = validator,
)

In [18]:
# Run a checkpoint
checkpoint_result = checkpoint_1.run()

Calculating Metrics:   0%|          | 0/31 [00:00<?, ?it/s]

# Data Docs

In [19]:
# Build data docs
context.build_data_docs()

{'local_site': 'file:///Users/nathanaelh/Desktop/untitled folder/gx/uncommitted/data_docs/local_site/index.html'}

# Conclusion

Based on the validation process using the Great Expectations Library, the dataset has undergone several validation processes, revealing critical insights into its quality and integrity. 

The validation process conducted on the dataset reveals a robust and consistent dataset structure after the data cleaning procedures. Each validation test returned successful results, affirming the integrity and reliability of the dataset.